# **Classificador de quadro de lombalgia com dados do [Kaggle](https://www.kaggle.com/datasets/sammy123/lower-back-pain-symptoms-dataset)**

###**Grupo:** Amanda Moraes, Luís Eduardo Alves, Tales Alves, Samuel Santos e Maria Eduarda Santos

###**Modelos testados:**
*   MLP com uma camada intermediária
*   MLP com duas camadas intermediárias

###**Algumas métricas de performance utilizadas:**

*   **Acurácia (total e categórica):** Métrica que descreve o desempenho do modelo como a razão entre o número de previsões corretas para o número total de previsões (no caso da categórica, são consideradas as predições de cada classe separadamente).

$$\frac{Positivos\ Verdadeiros\ +\ Negativos\ Verdadeiros}{(Positivos\ Verdadeiros\ +\ Positivos\ Falsos\ +\ Negativos\ Verdadeiros\ +\ Negativos\ Falsos)}$$

*   **Precisão:** Razão entre a quantidade de positivos verdadeiros e a quantidade total de positivos obtida.

$$\frac{Positivos\ Verdadeiros}{(Positivos\ Verdadeiros + Positivos\ Falsos)}$$

*   **Recall:** Razão entre a quantidade de positivos verdadeiros de uma classe e o total de amostras pertecentes à classe (podendo ser positivos verdadeiros e/ou negativos falsos).

$$\frac{Positivos\ Verdadeiros}{(Positivos\ Verdadeiros + Negativos\ Falsos)}$$

*   **F1-score**: Média harmônica entre a precisão e a recall.

$$\frac{2}{\frac{1}{Precisão} + \frac{1}{Recall}}$$

###**Algumas parâmetros considerados:**
*   **Quantidade de unidades por camada:** quantidade de neurônios por camada das redes neurais artificiais.

*   **Função de ativação:**
    * A **ReLu**, por exemplo, é amplamente aplicada a modelos deep porque seus gradientes são sempre 0 ou 1 (para qualquer valor) - o que evita em parte o problema de 'vanishing gradients'
    * A **Sigmoide** (logistic) tem gradientes próximos de zero para valores muito baixos ou muito altos - o que pode levar ao cenário de 'vanishing gradients' - e possui gradientes pequenos no geral (de 0 a 0.25)

*   **Taxa de aprendizagem:** Constante multiplicativa que interfere na velocidade de atualização dos parâmetros das redes: se for muito grande pode fazer o erro da rede oscilar e dificultar a convergência, se for muito pequeno pode atrasar o treinamento.

## 📂 **Dependências**

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from numpy import ndarray
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from datetime import datetime
from packaging import version

from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
import io
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.utils import shuffle
import random

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 133 kB 24.9 MB/s 


In [ ]:
import keras_tuner as kt

## 🔢 **Carregamento e processamento da base de dados**
Nesta seção, a base de dados é carregada, seus dados são normalizados e ela é dividida em sets de treino, teste e validação.

### Carregando base de dados

In [ ]:
#loading the dataset
from google.colab import files
uploaded = files.upload()

Saving Dataset_spine.csv to Dataset_spine (1).csv


### Analisando a base

In [ ]:
#decoding the files as uploaded will be a dictionary of keys (the file names) and values (the encoded file objects)
df = pd.read_csv(io.StringIO(uploaded['Dataset_spine.csv'].decode('utf-8')))
df.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Class_att,Unnamed: 13
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,0.744503,12.5661,14.5386,15.30468,-28.658501,43.5123,Abnormal,NaN
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,0.415186,12.8874,17.5323,16.78486,-25.530607,16.1102,Abnormal,NaN
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,0.474889,26.8343,17.4861,16.65897,-29.031888,19.2221,Abnormal,Prediction is done by using binary classificat...
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,0.369345,23.5603,12.7074,11.42447,-30.470246,18.8329,Abnormal,NaN
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,0.543360,35.4940,15.9546,8.87237,-16.378376,24.9171,Abnormal,NaN


In [ ]:
# The last column has metadata about the dataset
df['Unnamed: 13'].value_counts()

Prediction is done by using binary classification.    1
Attribute1  = pelvic_incidence  (numeric)             1
Attribute2 = pelvic_tilt (numeric)                    1
Attribute3 = lumbar_lordosis_angle (numeric)          1
Attribute4 = sacral_slope (numeric)                   1
Attribute5 = pelvic_radius (numeric)                  1
Attribute6 = degree_spondylolisthesis (numeric)       1
 Attribute7= pelvic_slope(numeric)                    1
 Attribute8= Direct_tilt(numeric)                     1
 Attribute9= thoracic_slope(numeric)                  1
 Attribute10= cervical_tilt(numeric)                  1
 Attribute11=sacrum_angle(numeric)                    1
 Attribute12= scoliosis_slope(numeric)                1
Attribute class {Abnormal, Normal}                    1
Name: Unnamed: 13, dtype: int64

In [ ]:
# Drop the metadata column
df.drop(columns=['Unnamed: 13'], inplace=True)
df.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Class_att
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,0.744503,12.5661,14.5386,15.30468,-28.658501,43.5123,Abnormal
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,0.415186,12.8874,17.5323,16.78486,-25.530607,16.1102,Abnormal
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,0.474889,26.8343,17.4861,16.65897,-29.031888,19.2221,Abnormal
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,0.369345,23.5603,12.7074,11.42447,-30.470246,18.8329,Abnormal
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,0.543360,35.4940,15.9546,8.87237,-16.378376,24.9171,Abnormal


In [ ]:
# Amount of records
len(df)

310

### Balanceamento da base

In [ ]:
# Quantidade de amostras por classe: há um desequilíbrio
df['Class_att'].value_counts()

Abnormal    210
Normal      100
Name: Class_att, dtype: int64

#### Abnormal class

In [ ]:
# Lista de indices das linhas que são da classe Abnormal
index_list_abnormal = list(df[df['Class_att'] == 'Abnormal'].index)

In [ ]:
# Listas de indices da classe Abnormal que devem compor os sets de treino, teste e validação
index_list_train_abnormal, index_list_test_abnormal = train_test_split(index_list_abnormal, test_size=0.3, random_state=23, shuffle=True)
index_list_test_abnormal, index_list_val_abnormal = train_test_split(index_list_test_abnormal, test_size=0.5, random_state=23, shuffle=True)

In [ ]:
print(f"Total size: {len(index_list_abnormal)} \nTrain size: {len(index_list_train_abnormal)} \nTest and val size: {len(index_list_test_abnormal)} and {len(index_list_val_abnormal)}")

Total size: 210 
Train size: 147 
Test and val size: 31 and 32


#### Normal class

In [ ]:
# Lista de indices das linhas que são da classe Normal
index_list_normal = list(df[df['Class_att'] == 'Normal'].index)

In [ ]:
# Listas de indices da classe Nnormal que devem compor os sets de treino, teste e validação
index_list_train_normal, index_list_test_normal = train_test_split(index_list_normal, test_size=0.3, random_state=23, shuffle=True)
index_list_test_normal, index_list_val_normal = train_test_split(index_list_test_normal, test_size=0.5, random_state=23, shuffle=True)

In [ ]:
print(f"Total size: {len(index_list_normal)} \nTrain size: {len(index_list_train_normal)} \nTest and val size: {len(index_list_test_normal)} and {len(index_list_val_normal)}")

Total size: 100 
Train size: 70 
Test and val size: 15 and 15


#### Aumentando a classe minoritária para os conjuntos de treino e validação
(que interferem no aprendizado do modelo)

In [ ]:
# Aumento da lista de indices de treino da classe Normal (minoritária) com replicações de amostras
dif_classes = len(index_list_train_abnormal) - len(index_list_train_normal)
index_list_train_normal.extend(index_list_train_normal[0:(dif_classes)])
print(f"Class Normal train set new size: {len(index_list_train_normal)}")

Class Normal train set new size: 140


In [ ]:
# Aumento da lista de indices de validação da classe Normal (minoritária) com replicações de amostras
dif_classes = len(index_list_val_abnormal) - len(index_list_val_normal)
index_list_val_normal.extend(index_list_val_normal[0:(dif_classes)])
print(f"Class Normal validation set new size: {len(index_list_val_normal)}")

Class Normal validation set new size: 30


#### Listas de indices finais dos sets de treino, teste e validação

In [ ]:
# treino
index_list_train = index_list_train_abnormal + index_list_train_normal
index_list_train = shuffle(index_list_train, random_state=23)

In [ ]:
# teste
index_list_test = index_list_test_abnormal + index_list_test_normal
index_list_test = shuffle(index_list_test, random_state=23)

In [ ]:
# validação
index_list_val = index_list_val_abnormal + index_list_val_normal
index_list_val = shuffle(index_list_val, random_state=23)

### Splitando a base em entradas e labels

In [ ]:
x = df[df.columns[:-1]]
x

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,0.744503,12.5661,14.5386,15.30468,-28.658501,43.5123
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,0.415186,12.8874,17.5323,16.78486,-25.530607,16.1102
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,0.474889,26.8343,17.4861,16.65897,-29.031888,19.2221
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,0.369345,23.5603,12.7074,11.42447,-30.470246,18.8329
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,0.543360,35.4940,15.9546,8.87237,-16.378376,24.9171
...,...,...,...,...,...,...,...,...,...,...,...,...
305,47.903565,13.616688,36.000000,34.286877,117.449062,-4.245395,0.129744,7.8433,14.7484,8.51707,-15.728927,11.5472
306,53.936748,20.721496,29.220534,33.215251,114.365845,-0.421010,0.047913,19.1986,18.1972,7.08745,6.013843,43.8693
307,61.446597,22.694968,46.170347,38.751628,125.670725,-2.707880,0.081070,16.2059,13.5565,8.89572,3.564463,18.4151
308,45.252792,8.693157,41.583126,36.559635,118.545842,0.214750,0.159251,14.7334,16.0928,9.75922,5.767308,33.7192


In [ ]:
y = pd.DataFrame(df[df.columns[-1]])
y

,Class_att
0,Abnormal
1,Abnormal
2,Abnormal
3,Abnormal
4,Abnormal
...,...
305,Normal
306,Normal
307,Normal
308,Normal


### One-hot-encode dataframe de labels

In [ ]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(y['Class_att'])
# Drop column B as it is now encoded
y = y.drop('Class_att',axis = 1)
# Join the encoded df
y = y.join(one_hot)
y

,Abnormal,Normal
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
305,0,1
306,0,1
307,0,1
308,0,1


### Normalizando a base de entradas

In [ ]:
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x.to_numpy())
x_scaled = pd.DataFrame(x_scaled, columns=x.columns)
x_scaled.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12
0,0.355688,0.519900,0.229180,0.250857,0.307461,0.025148,0.744554,0.186396,0.610506,0.845115,0.156861,0.977797
1,0.124501,0.296783,0.098578,0.144629,0.476649,0.036365,0.413783,0.197208,0.854170,0.996300,0.230878,0.243812
2,0.411666,0.513932,0.322995,0.307661,0.386097,0.017523,0.473750,0.666533,0.850409,0.983442,0.148026,0.327166
3,0.416151,0.557414,0.271260,0.289436,0.341826,0.051838,0.367741,0.556360,0.461461,0.448790,0.113989,0.316741
4,0.227272,0.289479,0.128129,0.247022,0.409579,0.044173,0.542524,0.957940,0.725757,0.188118,0.447450,0.479711


In [ ]:
x = x_scaled

### Gerando os sets de treino, teste e validação

In [ ]:
# treino
x_train, y_train = x.iloc[index_list_train], y.iloc[index_list_train]

In [ ]:
# teste
x_test, y_test = x.iloc[index_list_test], y.iloc[index_list_test]

In [ ]:
# validação
x_val, y_val = x.iloc[index_list_val], y.iloc[index_list_val]

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

((287, 12), (287, 2), (46, 12), (46, 2), (62, 12), (62, 2))

## 🧠 **Aproximação 1: Shallow MLP**

Utiliza 1 camada escondida e uma camada de saída com 2 neurônios (1 para cada classe).

### 🧰 **Construção e treinamento dos modelos**

In [ ]:
# Algumas constantes utilizadas
OUTPUT_NEURONS = 2
BATCH_SIZE = 4
SHUFFLE_SEED = 23

In [ ]:
# Preparação dos dados (train, test e val sets)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))


train_batches = train_dataset.shuffle(len(x_train), seed=SHUFFLE_SEED).batch(BATCH_SIZE)
val_batches = val_dataset.shuffle(len(x_val), seed=SHUFFLE_SEED).batch(BATCH_SIZE)
test_batches = test_dataset.batch(BATCH_SIZE)

#### 🚀 **Hyperparameters Tuning e Treinamento dos modelos**

Etapa implementada utilizando o Keras Tuner Hyperband para variar hiperparâmetros do modelo e treiná-lo com diferentes combinações de valores.

> "O algoritmo de ajuste Hyperband usa alocação adaptável de recursos e parada antecipada para convergir rapidamente em um modelo de alto desempenho. Isso é feito usando um suporte de estilo de campeonato esportivo. O algoritmo treina um grande número de modelos por algumas épocas e leva apenas a metade dos modelos com melhor desempenho para a próxima rodada."



In [ ]:
# Definição do modelo e das variações de parâmetros
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(
      hp.Choice('dense_units', [8, 16, 32, 64]),
      activation=hp.Choice('dense_activation', ['relu', 'sigmoid'])))
  model.add(keras.layers.Dense(OUTPUT_NEURONS, activation='softmax'))
  model.compile(optimizer=keras.optimizers.SGD(hp.Choice('learning_rate', values=[1e-2, 1e-1, 0.5])),
                loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
# Instanciação do Hyperband Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    hyperband_iterations=3,
    overwrite=True,
    seed=SHUFFLE_SEED)

In [ ]:
# Busca por combinações ótimas de parâmetros e treinamento de modelos
tuner.search(train_batches,
             validation_data=val_batches,
             epochs=50,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=7)])

Trial 22 Complete [00h 00m 00s]
val_accuracy: 0.5483871102333069

Best val_accuracy So Far: 0.774193525314331
Total elapsed time: 00h 00m 24s
INFO:tensorflow:Oracle triggered exit


### ✅ **Avaliando resultados dos modelos**

In [ ]:
# Elege os 3 melhores modelos (melhores combinações de hiperparâmetros)
best_models = tuner.get_best_models(3)
best_models_hyperparameters = tuner.get_best_hyperparameters(3)

In [ ]:
# Descreve e avalia cada modelo com o test set
for index in range(len(best_models)):
  # Seleciona modelo
  model = best_models[index]
  model_hp = best_models_hyperparameters[index]

  # Exibe descrição de seus hiperparâmetros
  print(f"Model #{index+1}")
  print(best_models_hyperparameters[index].get_config()['values'])
  print('\n')

  # Exibe 'classification report' do modelo com algumas métricas
  predictions = model.predict(x_test)
  y_pred = np.argmax(predictions, axis=1)
  y_expected = np.argmax(y_test.to_numpy(), axis=1)
  print(classification_report(y_expected, y_pred), '\n')

  # Exibe acurácia categórica do modelo
  matrix = confusion_matrix(y_expected, y_pred)
  print(f"Confusion matrix:\n{matrix}\n")
  cat_accuracy = matrix.diagonal()/matrix.sum(axis=1)
  for i in range(len(matrix.diagonal()/matrix.sum(axis=1))):
    print("{} class accuracy: {:.2f}%".format(i, (cat_accuracy[i]*100)))
  print('\n\n\n')

Model #1
{'dense_units': 8, 'dense_activation': 'relu', 'learning_rate': 0.5, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}


              precision    recall  f1-score   support

           0       0.70      0.61      0.66        31
           1       0.37      0.47      0.41        15

    accuracy                           0.57        46
   macro avg       0.54      0.54      0.53        46
weighted avg       0.59      0.57      0.58        46
 

Confusion matrix:
[[19 12]
 [ 8  7]]

0 class accuracy: 61.29%
1 class accuracy: 46.67%




Model #2
{'dense_units': 32, 'dense_activation': 'relu', 'learning_rate': 0.01, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}


              precision    recall  f1-score   support

           0       0.71      0.55      0.62        31
           1       0.36      0.53      0.43        15

    accuracy                           0.54        46
   macro avg       0.54      0.54

### 📕 **Resumo**

| Parâmetros variados | Valores considerados |
|--- |--- |
| Quantidade de unidades da camada intermediária | [ 8, 16, 32, 64 ] |
| Função de ativação da camada intermediária | [ sigmoid, relu ] |
| Taxa de aprendizagem | [ 0.01, 0.1, 0.5 ] |

---

| Parâmetros | Modelo 🥇 | Modelo 🥈 | Modelo 🥉
|--- |--- |--- |--- |
| Quantidade de unidades da camada intermediária | 8 | 32 | 64 |
| Função de ativação da camada intermediária | relu | relu | relu |
| Taxa de aprendizagem | 0.5 | 0.01 | 0.1 |

---

| Tendências de ganhos de desempenho |
|--- |
|Uso da função de ativação relu na camada intermediária|
|Taxa de aprendizagem com valores mais altos|

## 🧠 **Aproximação 2: Deep MLP**
2 *hidden layers* e uma camada de saída com uma unidade para cada uma das 2 classes do problema.

### 🧰 **Construindo o modelo**

In [ ]:
# Algumas constantes utilizadas
OUTPUT_NEURONS = 2
BATCH_SIZE = 4
SHUFFLE_SEED = 23

In [ ]:
# Preparação dos dados (train, test e val sets)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))


train_batches = train_dataset.shuffle(len(x_train), seed=SHUFFLE_SEED).batch(BATCH_SIZE)
val_batches = val_dataset.shuffle(len(x_val), seed=SHUFFLE_SEED).batch(BATCH_SIZE)
test_batches = test_dataset.batch(BATCH_SIZE)

#### 🚀 **Hyperparameters Tuning e Treinamento dos modelos**

Etapa implementada utilizando o Keras Tuner Hyperband para variar hiperparâmetros do modelo e treiná-lo com diferentes combinações de valores.

> "O algoritmo de ajuste Hyperband usa alocação adaptável de recursos e parada antecipada para convergir rapidamente em um modelo de alto desempenho. Isso é feito usando um suporte de estilo de campeonato esportivo. O algoritmo treina um grande número de modelos por algumas épocas e leva apenas a metade dos modelos com melhor desempenho para a próxima rodada."

In [ ]:
# Definição do modelo e das variações de parâmetros
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(
      hp.Choice('dense_1_units', [8, 16, 32, 64]),
      activation=hp.Choice('dense_1_activation', ['relu', 'sigmoid'])))
  model.add(keras.layers.Dense(
      hp.Choice('dense_2_units', [8, 16, 32, 64]),
      activation=hp.Choice('dense_2_activation', ['relu', 'sigmoid'])))
  model.add(keras.layers.Dense(OUTPUT_NEURONS, activation='softmax'))
  model.compile(optimizer=keras.optimizers.SGD(hp.Choice('learning_rate', values=[1e-2, 1e-1, 0.5])),
                loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
# Instanciação do Hyperband Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    hyperband_iterations=3,
    overwrite=True,
    seed=SHUFFLE_SEED)

In [ ]:
# Busca por combinações ótimas de parâmetros e treinamento de modelos
tuner.search(train_batches,
             validation_data=val_batches,
             epochs=50,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=7)])

Trial 252 Complete [00h 00m 07s]
val_accuracy: 0.8225806355476379

Best val_accuracy So Far: 0.8387096524238586
Total elapsed time: 00h 09m 59s
INFO:tensorflow:Oracle triggered exit


### ✅ **Avaliando resultados dos modelos**

In [ ]:
# Elege os 3 melhores modelos (melhores combinações de hiperparâmetros)
best_models = tuner.get_best_models(3)
best_models_hyperparameters = tuner.get_best_hyperparameters(3)

In [ ]:
# Descreve e avalia cada modelo com o test set
for index in range(len(best_models)):
  # Seleciona modelo
  model = best_models[index]
  model_hp = best_models_hyperparameters[index]

  # Exibe descrição de seus hiperparâmetros
  print(f"Model #{index+1}")
  print(best_models_hyperparameters[index].get_config()['values'])
  print('\n')

  # Exibe 'classification report' do modelo com algumas métricas
  predictions = model.predict(x_test)
  y_pred = np.argmax(predictions, axis=1)
  y_expected = np.argmax(y_test.to_numpy(), axis=1)
  print(classification_report(y_expected, y_pred), '\n')

  # Exibe acurácia categórica do modelo
  matrix = confusion_matrix(y_expected, y_pred)
  print(f"Confusion matrix:\n{matrix}\n")
  cat_accuracy = matrix.diagonal()/matrix.sum(axis=1)
  for i in range(len(matrix.diagonal()/matrix.sum(axis=1))):
    print("{} class accuracy: {:.2f}%".format(i, (cat_accuracy[i]*100)))
  print('\n\n\n')

Model #1
{'dense_1_units': 16, 'dense_1_activation': 'relu', 'dense_2_units': 32, 'dense_2_activation': 'relu', 'learning_rate': 0.1, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0229'}


              precision    recall  f1-score   support

           0       0.87      0.65      0.74        31
           1       0.52      0.80      0.63        15

    accuracy                           0.70        46
   macro avg       0.70      0.72      0.69        46
weighted avg       0.76      0.70      0.71        46
 

Confusion matrix:
[[20 11]
 [ 3 12]]

0 class accuracy: 64.52%
1 class accuracy: 80.00%




Model #2
{'dense_1_units': 8, 'dense_1_activation': 'sigmoid', 'dense_2_units': 64, 'dense_2_activation': 'relu', 'learning_rate': 0.1, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0239'}


              precision    recall  f1-score   support

           0       0.83   

### 📕 **Resumo**

| Parâmetros variados | Valores considerados |
|--- |--- |
| Quantidade de unidades das 2 camadas intermediária | [ 8, 16, 32, 64 ] |
| Função de ativação das 2 camadas intermediárias | [ sigmoid, relu ] |
| Taxa de aprendizagem | [ 0.01, 0.1, 0.5 ] |

---

| Parâmetros | Modelo 🥇 | Modelo 🥈 | Modelo 🥉
|--- |--- |--- |--- |
| Quantidade de unidades da camada intermediária 1 | 16 | 8 | 64 |
| Função de ativação da camada intermediária 1 | relu | sigmoid | relu |
| Quantidade de unidades da camada intermediária 2 | 64 | 32 | 32 |
| Função de ativação da camada intermediária 2 | relu | relu | relu |
| Taxa de aprendizagem | 0.1 | 0.1 | 0.1 |

---

| Tendências de ganhos de desempenho |
|--- |
|Aumento da quantidade de unidades intermediárias|
|Uso de de ReLu nas camadas intermediárias|
|Taxa de aprendizagem com valor intermediário (não atrasa nem dificulta a convergência)|

## 🧠 **Aproximação 3: DeepER MLP**
3 *hidden layers* e uma camada de saída com uma unidade para cada uma das 2 classes do problema.

### 🧰 **Construindo o modelo**

In [ ]:
# Algumas constantes utilizadas
OUTPUT_NEURONS = 2
BATCH_SIZE = 4
SHUFFLE_SEED = 23

In [ ]:
# Preparação dos dados (train, test e val sets)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))


train_batches = train_dataset.shuffle(len(x_train), seed=SHUFFLE_SEED).batch(BATCH_SIZE)
val_batches = val_dataset.shuffle(len(x_val), seed=SHUFFLE_SEED).batch(BATCH_SIZE)
test_batches = test_dataset.batch(BATCH_SIZE)

#### 🚀 **Hyperparameters Tuning e Treinamento dos modelos**

Etapa implementada utilizando o Keras Tuner Hyperband para variar hiperparâmetros do modelo e treiná-lo com diferentes combinações de valores.

> "O algoritmo de ajuste Hyperband usa alocação adaptável de recursos e parada antecipada para convergir rapidamente em um modelo de alto desempenho. Isso é feito usando um suporte de estilo de campeonato esportivo. O algoritmo treina um grande número de modelos por algumas épocas e leva apenas a metade dos modelos com melhor desempenho para a próxima rodada."

In [ ]:
# Definição do modelo e das variações de parâmetros
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(
      hp.Choice('dense_1_units', [8, 16, 32, 64]),
      activation=hp.Choice('dense_1_activation', ['relu', 'sigmoid'])))
  model.add(keras.layers.Dense(
      hp.Choice('dense_2_units', [8, 16, 32, 64]),
      activation=hp.Choice('dense_2_activation', ['relu', 'sigmoid'])))
  model.add(keras.layers.Dense(
      hp.Choice('dense_3_units', [8, 16, 32, 64]),
      activation=hp.Choice('dense_3_activation', ['relu', 'sigmoid'])))
  model.add(keras.layers.Dense(OUTPUT_NEURONS, activation='softmax'))
  model.compile(optimizer=keras.optimizers.SGD(hp.Choice('learning_rate', values=[1e-2, 1e-1, 0.5])),
                loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
# Instanciação do Hyperband Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    hyperband_iterations=3,
    overwrite=True,
    seed=SHUFFLE_SEED)

In [ ]:
# Busca por combinações ótimas de parâmetros e treinamento de modelos
tuner.search(train_batches,
             validation_data=val_batches,
             epochs=50,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=7)])

Trial 762 Complete [00h 00m 11s]
val_accuracy: 0.5322580933570862

Best val_accuracy So Far: 0.8548387289047241
Total elapsed time: 00h 31m 04s
INFO:tensorflow:Oracle triggered exit


### ✅ **Avaliando resultados dos modelos**

In [ ]:
# Elege os 3 melhores modelos (melhores combinações de hiperparâmetros)
best_models = tuner.get_best_models(3)
best_models_hyperparameters = tuner.get_best_hyperparameters(3)

In [ ]:
# Descreve e avalia cada modelo com o test set
for index in range(len(best_models)):
  # Seleciona modelo
  model = best_models[index]
  model_hp = best_models_hyperparameters[index]

  # Exibe descrição de seus hiperparâmetros
  print(f"Model #{index+1}")
  print(best_models_hyperparameters[index].get_config()['values'])
  print('\n')

  # Exibe 'classification report' do modelo com algumas métricas
  predictions = model.predict(x_test)
  y_pred = np.argmax(predictions, axis=1)
  y_expected = np.argmax(y_test.to_numpy(), axis=1)
  print(classification_report(y_expected, y_pred), '\n')

  # Exibe acurácia categórica do modelo
  matrix = confusion_matrix(y_expected, y_pred)
  print(f"Confusion matrix:\n{matrix}\n")
  cat_accuracy = matrix.diagonal()/matrix.sum(axis=1)
  for i in range(len(matrix.diagonal()/matrix.sum(axis=1))):
    print("{} class accuracy: {:.2f}%".format(i, (cat_accuracy[i]*100)))
  print('\n\n\n')

Model #1
{'dense_1_units': 8, 'dense_1_activation': 'relu', 'dense_2_units': 32, 'dense_2_activation': 'relu', 'dense_3_units': 32, 'dense_3_activation': 'relu', 'learning_rate': 0.1, 'tuner/epochs': 34, 'tuner/initial_epoch': 12, 'tuner/bracket': 3, 'tuner/round': 2, 'tuner/trial_id': '0445'}


              precision    recall  f1-score   support

           0       0.82      0.90      0.86        31
           1       0.75      0.60      0.67        15

    accuracy                           0.80        46
   macro avg       0.79      0.75      0.76        46
weighted avg       0.80      0.80      0.80        46
 

Confusion matrix:
[[28  3]
 [ 6  9]]

0 class accuracy: 90.32%
1 class accuracy: 60.00%




Model #2
{'dense_1_units': 8, 'dense_1_activation': 'relu', 'dense_2_units': 16, 'dense_2_activation': 'sigmoid', 'dense_3_units': 64, 'dense_3_activation': 'relu', 'learning_rate': 0.1, 'tuner/epochs': 34, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}


         

### 📕 **Resumo**

| Parâmetros variados | Valores considerados |
|--- |--- |
| Quantidade de unidades das 2 camadas intermediária | [ 8, 16, 32, 64 ] |
| Função de ativação das 2 camadas intermediárias | [ sigmoid, relu ] |
| Taxa de aprendizagem | [ 0.01, 0.1, 0.5 ] |

---

| Parâmetros | Modelo 🥇 | Modelo 🥈 | Modelo 🥉
|--- |--- |--- |--- |
| Quantidade de unidades da camada intermediária 1 | 8 | 8 | 64 |
| Função de ativação da camada intermediária 1 | relu | relu | relu |
| Quantidade de unidades da camada intermediária 2 | 16 | 32 | 32 |
| Função de ativação da camada intermediária 2 | relu | sigmoid | sigmoid |
| Quantidade de unidades da camada intermediária 3 | 64 | 32 | 64 |
| Função de ativação da camada intermediária 3 | relu | relu | sigmoid |
| Taxa de aprendizagem | 0.1 | 0.1 | 0.1 |

---

| Tendências de ganhos de desempenho |
|--- |
|Quantidade intermediária de unidades intermediárias|
|Uso de de ReLu nas camadas intermediárias|
|Taxa de aprendizagem com valor intermediário (não atrasa nem dificulta a convergência)|

## 🔚 **Conclusão**

Como visto ao longo dos experimentos, múltiplos modelos podem ser desenvolvidos para um problema e evidenciar tendências de ganhos de desempenho para uma tarefa como a de classificação. Alguns modelos e resultados obtidos estão registrados abaixo.


Método | Acurácia
--- | ---
Shallow MLP | 61% 
Deep MLP | 76%
Deeper MLP | 80%


Os resultados mostram a importância dos ajustes dos hiperparâmetros das redes neurais: eles são fatores determinantes para definir a topologia da rede e, consequentemente, seu desempenho e poder de processamento com relação aos dados utilizados. A utilização do Tuner foi extremamente importante no no projeto, conseguimos avaliar de forma sistemática e exaustiva diversas possibilidades de redes, competindo entre si, de forma que encontramos as que melhor se adaptaram ao problema em questão: neste caso, além das tendências de ganho percebidas nas métricas de cada MLP especificamente, também foi notado que as redes neurais mais profundas - ou o aumento de camadas intermediárias - performaram melhor para a classificação dos dados utilizados.